 ## Расчет процента оплат среди групп пользователей с разным уровнем сложности

In [45]:
import pandas as pd
import re 
events_df_all = pd.read_csv('events.csv',',')
purchase_df_all = pd.read_csv('purchase.csv',',')
purchase_df_all['event_type'] = 'purchase'
events_df_all = events_df_all.rename(columns = {"id": "event_id"})
purchase_df_all = purchase_df_all.rename(columns = {"id": "purchase_id"})

# Creating df with events after 01 Jan 2018 and before 01 Jan 2019

cond = (events_df_all.start_time>='2018-01-01')&(events_df_all.start_time<'2019-01-01')&(events_df_all.event_type=='registration')
registered = events_df_all[cond]['user_id'].to_list()
events_df = events_df_all[events_df_all.user_id.isin(registered)]
purchase_df = purchase_df_all[purchase_df_all.user_id.isin(registered)]

# Check of levels to create groups
print("Типы уровней: {}".format(events_df['selected_level'].unique()))

# Creating sets of users based on selected level

users_easy_level = events_df[events_df["selected_level"]=="easy"]["user_id"].unique()
users_medium_level = events_df[events_df["selected_level"]=="medium"]["user_id"].unique()
users_hard_level = events_df[events_df["selected_level"]=="hard"]["user_id"].unique()

set_users_easy_level = set(users_easy_level)
set_users_medium_level = set(users_medium_level)
set_users_hard_level = set(users_hard_level)

# Calculating percentage of purchases for level Easy 

purchase_df_easy_level = purchase_df[purchase_df["user_id"].isin(set_users_easy_level)]
events_df_easy_level = events_df[events_df["user_id"].isin(set_users_easy_level)]
users_purchase_easy = purchase_df_easy_level["user_id"].nunique()
users_total_easy = events_df_easy_level["user_id"].nunique()
percent_payment_easy = users_purchase_easy / users_total_easy 
print(
    "Процент оплаты среди пользователей выбравших лёгкий уровень: {:.2%}".format(
        percent_payment_easy)
)

# Calculating percentage of purchases for level Medium 

purchase_df_medium_level = purchase_df[purchase_df["user_id"].isin(set_users_medium_level)]
events_df_medium_level = events_df[events_df["user_id"].isin(set_users_medium_level)]
users_purchase_medium = purchase_df_medium_level["user_id"].nunique()
users_total_medium = events_df_medium_level["user_id"].nunique()
percent_payment_medium = users_purchase_medium / users_total_medium 
print(
    "Процент оплаты среди пользователей выбравших средний уровень: {:.2%}".format(
        percent_payment_medium)
)

# Calculating percentage of purchases for level Hard 

purchase_df_hard_level = purchase_df[purchase_df["user_id"].isin(set_users_hard_level)]
events_df_hard_level = events_df[events_df["user_id"].isin(set_users_hard_level)]
users_purchase_hard = purchase_df_hard_level["user_id"].nunique()
users_total_hard = events_df_hard_level["user_id"].nunique()
percent_payment_hard = users_purchase_hard / users_total_hard 
print(
    "Процент оплаты среди пользователей выбравших сложный уровень: {:.2%}".format(
        percent_payment_hard)
)



Типы уровней: [nan 'medium' 'hard' 'easy']
Процент оплаты среди пользователей выбравших лёгкий уровень: 7.72%
Процент оплаты среди пользователей выбравших средний уровень: 20.86%
Процент оплаты среди пользователей выбравших сложный уровень: 35.39%


### Вопрос: Eсть ли зависимость между выбранным уровнем сложности и вероятностью оплаты? 
### Ответ: Да, есть прямая зависимость между уровнем сложности и вероятностью оплаты: чем выше уровень сложности, тем выше вероятность оплаты

## Расчет среднего времени между выбором уровня сложности и покупкой

In [68]:
# Creating merged Df for EASY level and changing date format

total_easy_df = pd.concat([purchase_df_easy_level,events_df_easy_level],sort=False)
total_easy_df["event_datetime"] = pd.to_datetime(total_easy_df["event_datetime"])
total_easy_df["start_time"] = pd.to_datetime(total_easy_df["start_time"])

# Creating merged Df for MEDIUM level and changing date format

total_medium_df = pd.concat([purchase_df_medium_level,events_df_medium_level],sort=False)
total_medium_df["event_datetime"] = pd.to_datetime(total_medium_df["event_datetime"])
total_medium_df["start_time"] = pd.to_datetime(total_medium_df["start_time"])

# Creating merged Df for HARD level and changing date format

total_hard_df = pd.concat([purchase_df_hard_level,events_df_hard_level],sort=False)
total_hard_df["event_datetime"] = pd.to_datetime(total_hard_df["event_datetime"])
total_hard_df["start_time"] = pd.to_datetime(total_hard_df["start_time"])

# Calculating average time between level choice and purchase for EASY level

purchase_easy_df = total_easy_df[total_easy_df['event_type'] == "purchase"]
purchase_easy_df = purchase_easy_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"})
level_choice_easy_df = total_easy_df[total_easy_df['event_type'] == "level_choice"]
level_choice_easy_df = level_choice_easy_df[["user_id","start_time"]].rename(columns={"start_time":"level_choice_time"})
purchase_level_easy = purchase_easy_df.merge(level_choice_easy_df, on="user_id", how="inner")

purchase_level_easy["timedelta"] = purchase_level_easy["purchase_time"] - purchase_level_easy["level_choice_time"]
print("В среднем, промежуток времени между выбором уровня и покупкой для лёгкого уровня составляет: {}".format(purchase_level_easy["timedelta"].mean()))

# Calculating average time between level choice and purchase for MEDIUM level

purchase_medium_df = total_medium_df[total_medium_df['event_type'] == "purchase"]
purchase_medium_df = purchase_medium_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"})
level_choice_medium_df = total_medium_df[total_medium_df['event_type'] == "level_choice"]
level_choice_medium_df = level_choice_medium_df[["user_id","start_time"]].rename(columns={"start_time":"level_choice_time"})
purchase_level_medium = purchase_medium_df.merge(level_choice_medium_df, on="user_id", how="inner")

purchase_level_medium["timedelta"] = purchase_level_medium["purchase_time"] - purchase_level_medium["level_choice_time"]
print("В среднем, промежуток времени между выбором уровня и покупкой для среднего уровня составляет: {}".format(purchase_level_medium["timedelta"].mean()))

# Calculating average time between level choice and purchase for HARD level

purchase_hard_df = total_hard_df[total_hard_df['event_type'] == "purchase"]
purchase_hard_df = purchase_hard_df[["user_id", "event_datetime"]].rename(
    columns={"event_datetime": "purchase_time"})
level_choice_hard_df = total_hard_df[total_hard_df['event_type'] == "level_choice"]
level_choice_hard_df = level_choice_hard_df[["user_id","start_time"]].rename(columns={"start_time":"level_choice_time"})
purchase_level_hard = purchase_hard_df.merge(level_choice_hard_df, on="user_id", how="inner")

purchase_level_hard["timedelta"] = purchase_level_hard["purchase_time"] - purchase_level_hard["level_choice_time"]
print("В среднем, промежуток времени между выбором уровня и покупкой для тяжёлого уровня составляет: {}".format(purchase_level_hard["timedelta"].mean()))



Промежуток времени между выбором уровня и покупкой для лёгкого уровня составляет: 3 days 14:58:52.941798941
Промежуток времени между выбором уровня и покупкой для среднего уровня составляет: 3 days 23:14:13.165118679
Промежуток времени между выбором уровня и покупкой для тяжёлого уровня составляет: 3 days 07:20:41.420814479


### Проходит, как минимум, три дня между этапом выбора уровня сложности и оплатой. Разница во времени между уровнями сложности не значительная и остается в рамках одного дня.

## Рассчет временного промежутка между регистрацией и оплатой для групп пользователей с разным уровнем сложности.


In [76]:
# Calculating average time between registration and purchase for EASY level

registration_easy_df = total_easy_df[total_easy_df['event_type'] == "registration"]
registration_easy_df = registration_easy_df[["user_id","start_time"]].rename(columns={"start_time":"registration_time"})
registration_to_purchase_easy = purchase_easy_df.merge(registration_easy_df, on="user_id", how="inner")

registration_to_purchase_easy["timedelta"] = registration_to_purchase_easy["purchase_time"] - registration_to_purchase_easy["registration_time"]
print("В среднем, промежуток времени между регистрацией и покупкой для легкого уровня составляет: {}".format(registration_to_purchase_easy["timedelta"].mean()))

# Calculating average time between registration and purchase for MEDIUM level

registration_medium_df = total_medium_df[total_medium_df['event_type'] == "registration"]
registration_medium_df = registration_medium_df[["user_id","start_time"]].rename(columns={"start_time":"registration_time"})
registration_to_purchase_medium = purchase_medium_df.merge(registration_medium_df, on="user_id", how="inner")

registration_to_purchase_medium["timedelta"] = registration_to_purchase_medium["purchase_time"] - registration_to_purchase_medium["registration_time"]
print("В среднем, промежуток времени между регистрацией и покупкой для среднего уровня составляет: {}".format(registration_to_purchase_medium["timedelta"].mean()))

# Calculating average time between registration and purchase for HARD level

registration_hard_df = total_hard_df[total_hard_df['event_type'] == "registration"]
registration_hard_df = registration_hard_df[["user_id","start_time"]].rename(columns={"start_time":"registration_time"})
registration_to_purchase_hard = purchase_hard_df.merge(registration_hard_df, on="user_id", how="inner")

registration_to_purchase_hard["timedelta"] = registration_to_purchase_hard["purchase_time"] - registration_to_purchase_hard["registration_time"]
print("В среднем, промежуток времени между регистрацией и покупкой для тяжелого уровня составляет: {}".format(registration_to_purchase_hard["timedelta"].mean()))



В среднем, промежуток времени между регистрацией и покупкой для легкого уровня составляет: 3 days 22:10:23.211640211
В среднем, промежуток времени между регистрацией и покупкой для среднего уровня составляет: 4 days 06:12:06.576883384
В среднем, промежуток времени между регистрацией и покупкой для тяжелого уровня составляет: 3 days 14:55:19.257918552


### Временный промежутки между регистрацией и покупкой для разный групп сложности отличаются. Больше всего времени между регистраций и покупкой проходит для среднего уровня  - 4 дня и 6 часов, для легкого уровня - 3 дня и 22 часа и для тяжелого уровня проходит меньше всего времени - 3 дня и 14 часов.